In [297]:
import numpy as np
import json
import color
from random import randint
from dataclasses import dataclass
import math
import importlib
from random import random

importlib.reload(color)
from color import randomColor, Color

In [298]:
# target = 'E:/effects/{}.effect'
target = '{}.effect'

In [299]:
lightcount = 100

In [300]:
def riseFall(fr0m, to, num):
    return np.concatenate((np.linspace(fr0m, to, num//2), np.linspace(to, fr0m, num//2 + num%2)))

def clamp(x, min, max):
    if x > max:
        return x

    if x < min:
        return min

    return x

In [301]:
def fade():
    result = dict()
    result['frame_delay_ms'] = 50

    x = np.arange(0, 100, 1)
    lights = -(x)*(x - 100)*255/(50**2)
    lights = np.reshape(lights, (len(lights), 1))
    lights = np.repeat(lights, lightcount, axis=1)
    lights = np.concatenate((lights, np.zeros((20, lightcount))))

    return lights, result, "Fade"


In [302]:
def travel():
    result = dict()
    result['frame_delay_ms'] = 50

    lights = np.zeros((500, lightcount))

    @dataclass
    class Thing:
        color: int
        offset: int
        position: int = 0

    things = [Thing(color=randomColor(), offset=randint(5, 15)) for _ in range(200)]
    things[0].offset = 0

    total = 0
    for thing in things:
        total += thing.offset
        thing.offset = total

    for i in range(np.size(lights, axis=0)):
        for thing in things:
            if thing.offset <= i:
                for j in range(0, thing.position):
                    lights[i, j] = thing.color

                thing.position += 1

                if thing.position >= lightcount:
                    thing.position = lightcount

    return lights, result, "Travel"


In [303]:
def movingPoints():
    result = dict()
    result['frame_delay_ms'] = 20

    @dataclass
    class ColorThing:
        color: Color
        x: float = 0
        v: float = 0.08

    colorA = Color.rgb(1, 0, 0)
    colorB = Color.rgb(169/255, 0, 191/255)

    colorThings = [
        ColorThing(color=colorA.hue(i/40), x = 4*i - 30) for i in range(40)
    ]

    a = 0.001

    lights = np.zeros((3000, lightcount)).astype(int)

    for i in range(np.size(lights, axis=0)):
        for thing in colorThings:
            l = thing.color.toHls()[1]
            if (thing.x < lightcount and thing.x >= 0):
                if int(thing.x) == thing.x:
                    lights[i, int(thing.x)] = thing.color
                elif thing.x < lightcount-1:
                    lights[(i, math.floor(thing.x))] = thing.color.lightness(l*(math.ceil(thing.x) - thing.x))
                    lights[(i, math.ceil(thing.x))] = thing.color.lightness(l*(thing.x - math.floor(thing.x)))
                else:
                    lights[i, 49] = thing.color.lightness(clamp(l*(1 + (-thing.x + lightcount)), 0, l))


            if (thing.x >= lightcount + 30):
                thing.x = 0

            thing.x += thing.v

    lights = lights[~np.all(lights == 0, axis=1)]

    return lights, result, "Moving points"


In [304]:
def fade2():
    result = dict()
    result['frame_delay_ms'] = 20

    x = np.arange(0, 100, 1)
    lights = -(x)*(x - 100)/(50**2)
    lights = np.reshape(lights, (len(lights), 1))
    lights = np.repeat(lights, lightcount, axis=1)
    lights = np.concatenate((lights, np.zeros((30, lightcount))))

    def toColor(color: Color):
        oldL = color.toHls()[1]

        def inner(l: float):
            nonlocal oldL
            return int(color.lightness(l*oldL))

        return np.vectorize(inner)

    lights = np.concatenate(
        tuple(toColor(Color.hls(i/20, 0.5, 1))(lights) for i in range(20))
    )

    return lights, result, "Fade 2"


In [305]:
def rainbow():
    result = dict()
    result['frame_delay_ms'] = 25

    frames = lightcount + 1
    lights = np.zeros((frames, lightcount))

    toColor = lambda color: np.vectorize(lambda h: int(color.hue(h)))

    for i in range(frames):
        lights[i,:] = np.roll(np.linspace(0, 1, lightcount), i)

    return toColor(Color.rgb(1, 0, 0))(lights), result, "Rainbow"

In [312]:
def carameldansen():
    result = dict()
    result['frame_delay_ms'] = 500

    colors = [
        Color.rgb(255, 0, 0),
        Color.rgb(0, 255, 0),
        Color.rgb(0, 0, 255),
        Color.rgb(255, 128, 0),
        Color.rgb(255, 0, 255),
        Color.rgb(255, 255, 0),
        Color.rgb(0, 255, 255),
    ]

    lights = [
        np.reshape(
            np.repeat(
                int(color),
                lightcount
            ),
            (1, lightcount)
        ) for color in colors
    ]

    return np.concatenate(lights, axis=0), result, "Carameldansen"

In [313]:
# def movingBalls():
#     result = dict()
#     result['frame_delay_ms'] = 20

#     @dataclass
#     class Ball:
#         r: float
#         x: float
#         y: float
#         color: int

#     balls = [Ball(r=random(4, 12), x=random(0, lightcount), y=random(3, 10), color=randomColor()) for _ in range(10)]



#     return lights, result, movingBalls.__name__

In [314]:
import interactiveSorting
importlib.reload(interactiveSorting)

from interactiveSorting import heapSort, bubbleSort

def sort():
    result = dict()
    result['frame_delay_ms'] = 70

    lights = np.zeros((4000, lightcount))

    lights[0,:] = np.random.rand(1, lightcount)

    myList = lights[0,:].tolist()

    row = 1
    def update():
        nonlocal row
        lights[row,:] = myList.copy()
        row += 1

    sortAlg = bubbleSort

    sortAlg(myList, update)

    for i in range(40):
        lights[row+i,:] = lights[row+i-1,:]

    lights = lights[~np.all(lights == 0, axis=1)]

    toColor = lambda color: np.vectorize(lambda h: int(color.lightness(0.5*h)))

    return toColor(Color.rgb(1,0,1))(lights), result, sortAlg.__name__+"2"


In [315]:
def peaks():
    result = dict()
    result['frame_delay_ms'] = 20
    frames = 100

    d = 8
    x = np.reshape(np.linspace(0, lightcount, lightcount), (1, lightcount))
    t = np.reshape(np.linspace(0, math.pi*2, frames), (frames, 1))

    a = np.exp(-((x % d) - d/2)**2)*(np.sin(t - math.pi/2) + 1) * 0.5
    b = np.exp(-(((x - d/2) % d) - d/2)**2)*(np.sin(t + 2/3*math.pi) + 1)*0.5

    toColor = lambda color: np.vectorize(lambda l: int(color.lightness(l*0.5)))

    startColor = Color.rgb(1, 0, 0)
    aWhole = np.concatenate([ toColor(startColor.hue(h))(a) for h in np.linspace(0, 1, 10) ], axis=0)
    bWhole = np.concatenate([ toColor(startColor.hue(h))(b) for h in np.linspace(0, 1, 10) ], axis=0)

    lights = aWhole + bWhole

    return lights, result, "Peaks"

In [316]:
lights, result, effectName = carameldansen()
lights = np.around(lights).astype(int)
result['light_count'] = lightcount

In [317]:
import effect_serializer
importlib.reload(effect_serializer)

with open(target.format(effectName), 'w') as f:
    effect_serializer.serialize(f, lights, result)